In [1]:
#default_exp modules/tf_helper/training_functools.py

# Training functools

This module provides custom training functions for tensorflow training. Those tends to be a tad faster than keras built-in counterparts, probably due to smaller overhead.

In [2]:
#export
from collections import defaultdict
from itertools import islice

import tensorflow as tf
from tqdm import tqdm


@tf.function
def train_step(x, y, model):
    assert isinstance(model, tf.keras.Model)
    assert model.optimizer is not None, "Model not compiled!"
    assert model.loss is not None, "Model not compiled!"
    mixed_precision = isinstance(
        model.optimizer, tf.keras.mixed_precision.experimental.LossScaleOptimizer
    )
    with tf.GradientTape() as tape:
        outs = model(x, training=True)
        outs = tf.cast(outs, tf.float32)
        loss = model.compiled_loss(y, outs, regularization_losses=model.losses)
        if mixed_precision:
            loss = model.optimizer.get_scaled_loss(loss)

    gradients = tape.gradient(loss, model.trainable_variables)
    if mixed_precision:
        gradients = model.optimizer.get_unscaled_gradients(gradients)

    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    model.compiled_metrics.update_state(y, outs)
    return outs

`train_step` is a lowest level function for training the model. Because it is decorated with `@tf.function`, `assert` statemets should not be executed during the runtime -- but only during the tracing. This is great, because it provides necessary testing without slowing down the runtime. This implementation provides mixed precision support -- also without slowing down the runtime in case if mixed precision is not used. Here is a minimal example of model training that demonstrates decaying loss.

In [ ]:
from modules.tf_helper.models import VGG, WRN
from modules.tf_helper.datasets import cifar
from itertools import islice
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

# model = VGG((32, 32, 3), 10, version=11)
model = WRN(16, 8, (32, 32, 3), 10)
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')],
)
data = cifar(shuffle_train=False)

losses = []
accurs = []
for x, y in tqdm(islice(data['train'], 100), total=100):
    outs = train_step(x, y, model)
    losses.append(model.metrics[0].result())
    accurs.append(model.metrics[1].result())
    for metric in model.metrics:
        metric.reset_states()

plt.subplot(211)
plt.plot(range(len(losses)), losses)
plt.subplot(212)
plt.plot(range(len(accurs)), accurs)

# UNKNOWN CMD ARGUMENTS: ['-f', 'C:\\Users\\gaha\\AppData\\Roaming\\jupyter\\runtime\\kernel-679624c9-fbbd-4148-b592-cc75a6994f68.json']
#   KNOWN CMD ARGUMENTS: {'gpu': None, 'no_memory_growth': False}
# SETTING MEMORY GROWTH!
# ResNet: unknown parameters: []


  0%|          | 0/100 [00:00<?, ?it/s]

In the example above we train the network for 100 iterations. The loss should start going down and accuracy should start raising.

In [ ]:
#export
def train_epoch(iterator, model, epoch_idx=0, steps=None, callbacks=(), use_pbar=None):
    for callback in callbacks:
        assert isinstance(callback, tf.keras.callbacks.Callback)
        callback.on_epoch_begin(epoch_idx)

    if use_pbar:
        pbar = use_pbar
    else:
        pbar = tqdm(total=steps, leave=True, ascii=True)

    for bidx, (x, y) in enumerate(islice(iterator, steps)):
        for callback in callbacks:
            assert isinstance(callback, tf.keras.callbacks.Callback)
            callback.on_train_batch_begin(bidx)

        outs = train_step(x, y, model)
        pbar.set_postfix(
            {m.name: m.result().numpy() for m in model.metrics}, refresh=False
        )

        for callback in callbacks:
            assert isinstance(callback, tf.keras.callbacks.Callback)
            callback.on_train_batch_end(bidx)
        pbar.update()

    if not use_pbar:
        pbar.close()

    for callback in callbacks:
        assert isinstance(callback, tf.keras.callbacks.Callback)
        callback.on_epoch_end(epoch_idx)

In [5]:
import time

t0 = time.time()
train_epoch(
    data["train"], model, steps=100, callbacks=[tf.keras.callbacks.Callback()]
)
print(f"TIME: {time.time() - t0}")

for metric in model.metrics:
    print(f"{metric.name} = {metric.result()}")

TIME: 13.22787880897522
loss = 3.7753608226776123
accuracy = 0.4429687559604645


In [6]:
#export

@tf.function
def valid_step(x, y, model):
    assert isinstance(model, tf.keras.Model)
    assert model.loss is not None, "Model not compiled!"

    outs = model(x, training=False)
    outs = tf.cast(outs, tf.float32)
    model.compiled_loss(y, outs)
    model.compiled_metrics.update_state(y, outs)
    return outs


def valid_epoch(iterator, model, epoch_idx=0, steps=None, callbacks=()):
    for bidx, (x, y) in enumerate(islice(iterator, steps)):
        for callback in callbacks:
            assert isinstance(callback, tf.keras.callbacks.Callback)
            callback.on_test_batch_begin(bidx)

        outs = valid_step(x, y, model)

        for callback in callbacks:
            assert isinstance(callback, tf.keras.callbacks.Callback)
            callback.on_test_batch_end(bidx)

In [7]:
valid_epoch(data["test"], model)
for m in model.metrics:
    print(f"{m.name}: {m.result()}")

loss: 3.009577512741089
accuracy: 0.4010087847709656


In [8]:
#export

def reset_metrics(model):
    results = {m.name: m.result().numpy() for m in model.metrics}
    for metric in model.metrics:
        metric.reset_states()
    return results


def fit(
    model,
    training_data,
    validation_data,
    steps_per_epoch=None,
    epochs=1,
    initial_epoch=0,
    callbacks=(),
):
    history = defaultdict(list)

    bpbar = tqdm(total=epochs, leave=True, ascii=True)
    for epoch_idx in range(initial_epoch, epochs):
        pbar = tqdm(total=steps_per_epoch, leave=True, ascii=True)
        train_epoch(
            training_data,
            model,
            epoch_idx=epoch_idx,
            steps=steps_per_epoch,
            callbacks=callbacks,
            use_pbar=pbar,
        )
        metrics = reset_metrics(model)
        for key, value in metrics.items():
            history[key].append(value)

        valid_epoch(
            validation_data,
            model,
            epoch_idx=epoch_idx,
            steps=steps_per_epoch,
            callbacks=callbacks,
        )
        metrics = reset_metrics(model)
        for key, value in metrics.items():
            history["val_" + key].append(value)

        metrics = reset_metrics(model)

        pbar.close()
        bpbar.set_postfix({key: value[-1] for key, value in history.items()})
        bpbar.update()
    return dict(history)

In [9]:
fit(model, data["train"], data["test"], steps_per_epoch=10, epochs=5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'loss': [3.0393364, 3.4806876, 3.415501, 3.3726249, 3.2958267],
 'accuracy': [0.40502492, 0.521875, 0.5210937, 0.553125, 0.5546875],
 'val_loss': [1.8310251, 1.4278662, 1.6661994, 1.7883714, 1.638818],
 'val_accuracy': [0.38632813, 0.5001953, 0.41308594, 0.39648438, 0.43046874]}

In [13]:
%run convert_notebooks.py

Creating modules\tf_helper\training_functools.py
